In [ ]:
import json
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification


In [ ]:
dataset = pd.read_csv("PD_sentiment_analysis.csv")

In [ ]:
# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("w11wo/indonesian-roberta-base-sentiment-classifier")

# Tokenize with truncation
tokenized_data = tokenizer(
    dataset["content"].tolist(),
    return_tensors="tf",
    padding=True,
    truncation=True,  # Add truncation
    max_length=512    # Max length to truncate to
)

# Convert labels
labels = np.array(dataset[" label"])

In [ ]:
checkpoint = 'w11wo/indonesian-roberta-base-sentiment-classifier'
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [ ]:
# Compile
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

In [ ]:
# Parameter
num_epochs = 15
batch_size = 32
validation_split = 0.2

# Training
history = model.fit(
    tokenized_data,
    labels,
    epochs=num_epochs,
    batch_size=batch_size,
    validation_split=validation_split
)

# Evaluate the model
loss, accuracy = model.evaluate(tokenized_data, labels)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

# Save model
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

Epoch 1/15


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 73s 73s/step - loss: 0.8718 - accuracy: 0.8000 - val_loss: 8.0771 - val_accuracy: 0.0000e+00
Epoch 2/15
1/1 [==============================] - 2s 2s/step - loss: 3.0314 - accuracy: 0.6000 - val_loss: 1.3054 - val_accuracy: 0.0000e+00
Epoch 3/15
1/1 [==============================] - 2s 2s/step - loss: 2.2477 - accuracy: 0.3000 - val_loss: 3.2905 - val_accuracy: 0.0000e+00
Epoch 4/15
1/1 [==============================] - 2s 2s/step - loss: 1.3658 - accuracy: 0.6000 - val_loss: 0.2636 - val_accuracy: 1.0000
Epoch 5/15
1/1 [==============================] - 2s 2s/step - loss: 2.4344 - accuracy: 0.1000 - val_loss: 8.1553 - val_accuracy: 0.0000e+00
Epoch 6/15
1/1 [==============================] - 2s 2s/step - loss: 1.7502 - accuracy: 0.3000 - val_loss: 0.3145 - val_accuracy: 0.6667
Epoch 7/15
1/1 [=================

('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.json',
 './fine_tuned_model/merges.txt',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')

In [ ]:
# Push the model to huggingface
# Load model
fine_tuned_model_path = "farizkuy/SA_fine_tuned"
model = TFAutoModelForSequenceClassification.from_pretrained(fine_tuned_model_path)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Teks panjang untuk prediksi (diulang untuk memastikan panjangnya lebih dari 512 token)
long_text = """
    Argumen yang bapak berikan kurang bagus karena tidak mencakup semua aspek yang diperlukan.
    Namun, saya menghargai usaha Anda untuk mencoba menyampaikan pendapat Anda.
    Dalam dunia yang penuh dengan informasi, penting untuk memiliki argumen yang kuat dan berbasis data.
    Misalnya, dalam kasus perubahan iklim, ada banyak bukti yang menunjukkan dampak negatif dari emisi karbon.
    Ini termasuk peningkatan suhu global, pencairan es di kutub, dan peningkatan frekuensi bencana alam.
    Selain itu, pendekatan holistik yang melibatkan semua pemangku kepentingan sangat penting.
    Di sisi lain, kita juga harus mempertimbangkan aspek ekonomi dari setiap kebijakan yang diambil.
    Beralih ke energi terbarukan mungkin mahal pada awalnya, tetapi dalam jangka panjang bisa menghemat biaya dan menciptakan lapangan kerja baru.
    Jadi, mari kita fokus pada solusi yang berkelanjutan dan menguntungkan semua pihak.
    Saya juga ingin menekankan pentingnya pendidikan dan kesadaran masyarakat mengenai isu-isu lingkungan.
    Tanpa partisipasi aktif dari masyarakat, perubahan yang signifikan sulit dicapai.
    Oleh karena itu, mari kita bersama-sama berusaha untuk membuat dunia ini tempat yang lebih baik untuk generasi mendatang.
    Perubahan iklim bukanlah masalah yang bisa diselesaikan oleh satu negara saja, tetapi membutuhkan kerjasama global.
    Setiap negara harus berkontribusi dalam upaya mengurangi emisi dan melindungi lingkungan.
    Argumen yang baik harus didukung oleh data yang akurat dan relevan. Misalnya, ...
""" * 11  # Mengulangi teks untuk memastikan panjangnya lebih dari 512 token

# Fungsi untuk memecah teks panjang menjadi segmen-segmen yang dapat diolah model
def split_text_into_segments(text, tokenizer, max_length=512):
    tokens = tokenizer(text, return_tensors="tf", padding=False, truncation=False)["input_ids"][0]
    num_tokens = len(tokens)
    segments = []
    for i in range(0, num_tokens, max_length):
        segment = tokens[i:i + max_length]
        segments.append(segment)
    return segments

# Memecah teks panjang menjadi segmen-segmen
segments = split_text_into_segments(long_text, tokenizer)

# Fungsi untuk melakukan prediksi sentiment pada segmen-segmen dan menggabungkan hasilnya
def predict_sentiment_for_segments(segments, model):
    predictions = []
    for segment in segments:
        inputs = {"input_ids": tf.expand_dims(segment, 0)}
        outputs = model(inputs)
        logits = outputs.logits
        probabilities = tf.nn.softmax(logits, axis=-1).numpy()[0]
        predictions.append(probabilities)
    return predictions

# Melakukan prediksi sentiment pada segmen-segmen
predictions = predict_sentiment_for_segments(segments, model)

# Menggabungkan hasil prediksi untuk menentukan sentiment keseluruhan
average_probabilities = np.mean(predictions, axis=0)

# Mendapatkan label sentiment
id2label = {0: "positive", 1: "neutral", 2: "negative"}
final_prediction = [{"label": label, "score": f"{prob:.3f}"} for label, prob in zip(id2label.values(), average_probabilities)]

# Mengonversi hasil ke format JSON dan mencetaknya
output_json = json.dumps(final_prediction, indent=4)
print(output_json)


config.json:   0%|          | 0.00/966 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/499M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at farizkuy/SA_fine_tuned.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/808k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/467k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

[
    {
        "label": "positive",
        "score": "0.041"
    },
    {
        "label": "neutral",
        "score": "0.651"
    },
    {
        "label": "negative",
        "score": "0.308"
    }
]


In [ ]:
# Save the JSON string to a file in the fine_tuned_model directory
file_path = f"{fine_tuned_model_path}/sentiment_analysis_pred.json"
with open(file_path, "w") as json_file:
    json_file.write(output_json)

print(f"Output saved to {file_path}")

Output saved to ./fine_tuned_model/sentiment_analysis_pred.json
